<a href="https://colab.research.google.com/github/tmehtiyev2019/llm-api-tm/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install Dependencies
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00


In [2]:
#@title Import necessary libraries
from typing import Optional
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_openai_fn_runnable,
    create_structured_output_chain,
    create_structured_output_runnable,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from typing import Sequence
from langchain.pydantic_v1 import BaseModel, Field
import pandas as pd
from google.colab import drive
from langchain.llms import OpenAI
import openai

In [3]:
#@title Configure the api key
# Import the required module for accessing Google Drive
from google.colab import drive

# Mount your Google Drive to access files stored in it
drive.mount('/content/drive')

# Define the path to the API key file stored in Google Drive
# Make sure to update this path to point to the actual location of your API key file
api_key_path = '/content/drive/My Drive/api_key.txt'

# Open the API key file and read its contents
# The 'with' statement ensures the file is properly closed after its suite finishes
with open(api_key_path, 'r') as file:
    # Read the API key from the file and strip any leading/trailing whitespace
    openai_api_key = file.read().strip()

# At this point, 'openai_api_key' contains your API key and can be used for authentication
# in your OpenAI API calls


Mounted at /content/drive


In [4]:
#@title Generate structred api content via function calling in LLMs using langchain

# Function to process user input and generate a response using a specified LLM model
def process_user_input(input_text, model):
    # Initialize the LLM with specified model, API key, and other parameters
    llm = OpenAI(
        api_key=openai_api_key,
        model_name=model,  # Model name can be changed as needed
        temperature=0.5,
        max_tokens=64
    )
    try:
        # Generating a response from the LLM using LangChain with the provided input
        response = llm.generate(
            prompts=[input_text],
            max_tokens=150  # Maximum number of tokens for the generated response
        )

        # Extracting the generated text from the response
        generated_text = response.generations[0][0].text
        return generated_text
    except Exception as e:
        # Handling any errors encountered during generation
        print(f"Error encountered: {e}")
        return None

# Define a prompt template for guiding the LLM's response
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class algorithm for extracting product information in structured formats."),
    ("human", "Use the given format to extract product information from the following input: {input}"),
    ("human", "Tip: Make sure to answer in the correct format"),
])

# Define a JSON schema for structured API request generation
json_schema = {
    "title": "APIRequest",
    "description": "Schema for creating a new product in the catalog.",
    "type": "object",
    "properties": {
        "method": {"type": "string", "enum": ["POST"]},
        "endpoint": {"type": "string", "const": "/products"},
        "description": {"type": "string"},
        "requestBody": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "price": {"type": "number"},
                "quantity": {"type": "integer"}
            },
            "required": ["name", "price", "quantity"]
        }
    },
    "required": ["method", "endpoint", "description", "requestBody"]
}

# Function to structure output from LLM response based on provided schema
def api_structure_output(model, llm_response, json_schema, prompt):
    # Initialize ChatOpenAI with specified model and temperature
    llm = ChatOpenAI(api_key=openai_api_key, model=model, temperature=0)

    # Create a runnable for structured output generation
    runnable = create_structured_output_runnable(json_schema, llm, prompt)

    # Invoke the runnable to get the structured API request
    api_request = runnable.invoke({"input": llm_response})
    return api_request

# Define a user input for generating complex product descriptions
user_input = "Generate a sentence as if you want to add details of products to your product catalog including its name, price, and quantity in stock. But these sentences should not be easy to understand. The formats should vary"

# DataFrame to store the results of multiple runs
results_df = pd.DataFrame(columns=["LLM Response", "API Structure Result from GPT-3.5", "API Structure Result from GPT-4", "API Structure Result from GPT-4-Turbo"])

# Generate responses and structure them
for _ in range(100):
    # Generate input for the API structure generation
    api_input = process_user_input(user_input, "text-davinci-003")

    try:
        # Generate structured API requests using different models
        api_request_3_5 = api_structure_output("gpt-3.5-turbo-1106", api_input, json_schema, prompt)
        api_request_4 = api_structure_output("gpt-4", api_input, json_schema, prompt)
        api_request_4_turbo = api_structure_output("gpt-4-1106-preview", api_input, json_schema, prompt)
    except Exception as e:
        # Handle any exceptions during the process
        print(f"Error encountered: {e}")

    # Append the results to the DataFrame
    results_df = results_df.append({
        "LLM Response": str(api_input),
        "API Structure Result from GPT-3.5": api_request_3_5,
        "API Structure Result from GPT-4": api_request_4,
        "API Structure Result from GPT-4-Turbo": api_request_4_turbo,
    }, ignore_index=True)

# Display



.

"These items are a must-have for any collection, with a price tag of $50 and a quantity of 15: the 'Fancy Fandango' product line."


<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = resul

Error encountered: Could not parse function call data: Expecting value: line 7 column 14 (char 157)


<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = resul

Error encountered: Could not parse function call data: Expecting ',' delimiter: line 1 column 337 (char 336)


<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-4-8c5a0041c49c>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = resul

In [5]:
#@title Print result
results_df

,LLM Response,API Structure Result from GPT-3.5,API Structure Result from GPT-4,API Structure Result from GPT-4-Turbo
0,".\n\n""A variety of widgets, varying in price f...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': 'http://example...","{'method': 'POST', 'endpoint': '/products', 'd..."
1,".\n\n""We have a limited quantity of the XYZ it...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': 'http://api.exa...","{'method': 'POST', 'endpoint': '/products', 'd..."
2,".\n\n""This delightful offering is priced at $2...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': 'http://example...","{'method': 'POST', 'endpoint': '/products', 'd..."
3,".\n\n""A must-have item at an unbeatable price,...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': 'http://example...","{'method': 'POST', 'endpoint': '/products', 'd..."
4,".\n\n""Inventory's stocked with a selection of ...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': '/products', 'd..."
...,...,...,...,...
95,".\n\n""We have a plethora of items available, i...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': 'http://api.exa...","{'method': 'POST', 'endpoint': '/products', 'd..."
96,"\n\n""In our selection, we have a limited quant...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': '/products', 'd..."
97,"\n\n""This item, a must-have for any home, is i...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': 'http://example...","{'method': 'POST', 'endpoint': '/products', 'd..."
98,".\n\n""This item, 'Cerulean Blue', is available...","{'method': 'POST', 'endpoint': '/products', 'd...","{'method': 'POST', 'endpoint': 'http://example...","{'method': 'POST', 'endpoint': '/products', 'd..."


In [6]:
#@title Export result file to excel
results_df.to_excel('output.xlsx', index=False)